In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:913: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [ ]:
import os
from google.colab import userdata
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
search_tool = SerperDevTool() #Enables agents to perform Google-like web searches via the Serper API, useful for real-time info retrieval.
scrape_tool = ScrapeWebsiteTool() #Lets an agent extract text/data from a website URL.

In [ ]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent employs statistical modeling and machine learning techniques to provide critical insights. Renowned for its proficiency in data analysis, the Data Analyst Agent serves as a pivotal resource for informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

# role="Data Analyst"
# → Defines the job title or persona of the agent.

# goal="Monitor and analyze market data..."
# → Sets the agent’s mission, i.e., analyze market data in real-time and predict trends.

# backstory="Specializing in financial markets..."
# → Adds context for the agent’s behavior and decisions (helpful for LLMs to respond in-character).

# verbose=True
# → The agent will give detailed output, helpful for debugging or explanation.

# allow_delegation=True
# → This agent can delegate tasks to other agents (if used in a multi-agent Crew).

# tools=[scrape_tool, search_tool]
# → Gives the agent abilities:

# scrape_tool: lets the agent extract live data from websites.

# search_tool: lets the agent perform web searches (e.g., using Serper).

In [ ]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies leveraging insights from the Data Analyst Agent.",
    backstory="Possessing a deep understanding of financial markets and quantitative analysis, this agent formulates and optimizes trading strategies. It assesses the performance of diverse approaches to identify the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)
# This agent would:
# Receive insights or raw data (possibly from another agent like the Data Analyst).
# Use tools to fetch additional information if needed.
# Design, evaluate, and refine trading strategies.
# Optionally collaborate with or delegate tasks to other agents.

In [ ]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Recommend optimal trade execution strategies based on approved trading plans.",
    backstory="Specializing in the analysis of timing, price, and logistical details of potential trades, this agent evaluates these factors to provide well-founded recommendations. Its expertise ensures that trades are executed efficiently and in alignment with the overall strategy.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

# The execution_agent is an AI agent that:
# Turns trade strategies into actionable execution plans
# Considers real-world factors like price and timing
# Uses online tools to stay updated
# Works in sync with other agents to complete a trading workflow

In [ ]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks associated with potential trading activities.",
    backstory="With extensive expertise in risk assessment models and market dynamics, this agent thoroughly examines the potential risks of proposed trades. It delivers comprehensive analyses of risk exposure and recommends safeguards to ensure that trading activities align with the firm's risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)
# Works with: Trading Strategy Developer and Execution Agent

# Purpose: Acts as a safeguard — ensuring that proposed trades are within acceptable risk levels

# Outputs: Warnings, risk ratings, mitigation plans, and adjustment recommendations



## Create Tasks

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for the selected stock ({stock_selection}). "
        "Employ statistical modeling and machine learning techniques to identify trends and predict market movements."
    ),
    expected_output=(
        "Generate insights and alerts regarding significant market opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

# description
# → Tells the agent what to do:
# "Continuously monitor and analyze market data for the selected stock ({stock_selection}). Employ statistical modeling and machine learning techniques to identify trends and predict market movements."
# {stock_selection} is a placeholder — you'll replace this with a specific stock like "AAPL" or "TSLA" at runtime.

# The agent is expected to fetch market data, apply stats/ML models, and find trends.

# expected_output:
# "Generate insights and alerts regarding significant market opportunities or threats for {stock_selection}."
# The output should include predictions, alerts, and insightful summaries about the stock’s movements.

# agent=data_analyst_agent
# → Links this task to the previously defined data_analyst_agent.

In [ ]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on insights from the Data Analyst and user-defined risk tolerance ({risk_tolerance}). "
        "Incorporate trading preferences ({trading_strategy_preference}) in the strategy development process."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

# description
# → Tells the agent what to do:
# and refine trading strategies based on insights from the Data Analyst and user-defined risk tolerance ({risk_tolerance}). Incorporate trading preferences ({trading_strategy_preference}) in the strategy development process."
# It relies on:

# Insights from the Data Analyst Agent
# A user’s risk tolerance (e.g., low, medium, high)
# A user's strategy preference (e.g., swing trading, day trading, momentum)

# expected_output
# → Specifies what to return:
# "A set of potential trading strategies for {stock_selection} that align with the user's risk tolerance."
# Outputs will be tailored to a specific stock ({stock_selection}), respecting the user’s profile and preferences.

# agent=trading_strategy_agent
# → Assigns the task to the Trading Strategy Developer Agent.

In [ ]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the optimal execution methods for {stock_selection}, "
        "considering current market conditions and pricing strategies."
    ),
    expected_output=(
        "Comprehensive execution plans detailing how and when to execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

# description
# → Instructs the agent:
# "Analyze approved trading strategies to determine the optimal execution methods for {stock_selection}, considering current market conditions and pricing strategies."
# The agent uses:

# The strategies from the Trading Strategy Agent
# Real-time market and price data
# Goal: find the best how and when to execute trades

# expected_output
# → What the agent should deliver:

# "Comprehensive execution plans detailing how and when to execute trades for {stock_selection}."
# Should include details like:
# Timing (e.g., trade at market open or during low volatility)
# Order type (e.g., limit vs. market order)
# Execution venue or platform

# agent=execution_agent
# → Binds this task to the Trade Advisor Agent.

In [ ]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks and recommend mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

# description
# → Tells the agent:

# "Evaluate the risks associated with the proposed trading strategies and execution plans for {stock_selection}. Provide a detailed analysis of potential risks and recommend mitigation strategies."
# The agent analyzes:

# The strategies built by the Trading Strategy Agent
# The execution plans from the Trade Advisor Agent
# Its job: identify risks and recommend safeguards

# expected_output
# → What the agent should return:
# "A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_selection}."
# This includes:

# Market risks (volatility, liquidity)
# Execution risks (slippage, timing issues)
# Strategy risks (overexposure, lack of diversification)
# Recommended mitigation strategies

# agent=risk_management_agent
# → Assigns the task to the Risk Advisor Agent.

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

# The Manager LLM in a CrewAI setup acts as the orchestrator or project manager of your AI agent team. It’s not just another agent — it’s the brain behind the coordination, deciding how the tasks flow, how outputs are passed between agents, and whether tasks are complete.

# What is the Manager LLM?
# It's a ChatGPT-like language model (e.g., gpt-3.5-turbo, gpt-4-turbo) that oversees the entire multi-agent workflow.

# It uses task descriptions, agent outputs, and context to:
# Interpret the current state
# Choose the right agent for the next step
# Route data between agents
# Decide when the job is done

# With a Manager LLM (in hierarchical mode):

# It first picks the Data Analyst Agent to analyze the stock.
# After that task completes, it passes the insights to the Strategy Agent.
# Then it tells the Execution Agent to plan execution based on the strategy.
# Finally, it sends the strategy + execution to the Risk Agent for validation.
# All this happens without you manually writing routing logic — the Manager LLM handles the entire workflow intelligently.

#  Why It’s Important
# Without the Manager LLM:
# You’d have to manually define the flow, inputs/outputs, and handle all logic in code.
# With it, CrewAI becomes modular, dynamic, and autonomous — able to work like a human team guided by a smart manager.


# tempertature:
# Use 0.0 - 0.3 for risk, execution, compliance, or analysis agents.

# Use 0.5 - 0.8 for strategy or manager agents to allow variation but stay practical.

# Use 1.0+ only if you're experimenting or asking the agents to generate creative financial ideas.

# Defines the execution logic or flow among tasks and agents. Options include:
# Process.sequential :	Tasks are executed one after another in the given order.
# Process.hierarchical :	The manager LLM controls the flow and can redirect, delegate, or adjust task execution based on outcomes. More flexible and intelligent.
# Process.parallel : Tasks can be executed simultaneously if agents are independent. Useful for batch processing.

In [ ]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'NVDA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Employ statistical modeling and machine learning techniques to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to generate insights and alerts regarding significant market opportunities or threats.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "The Data Analyst should utilize statistical modeling and machine learning techniques to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
To continuously monitor and analyze market data for AAPL, I need to gather real-time data from reliable financial news websites and stock market data platforms. I will begin by searching

KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
Markdown(result)
# is used in Jupyter Notebook or Google Colab environments to render rich, formatted Markdown output from a string.

Risk Analysis Report for Proposed Trading Strategies for AAPL:

Introduction:
The following risk analysis report aims to evaluate the potential risks associated with the proposed trading strategies for AAPL and provide recommendations for risk mitigation strategies. The strategies include Long-Term Investment Strategy, Options Trading Strategy, Swing Trading Strategy, and Diversification Strategy. Each strategy presents unique risks that should be carefully considered to optimize returns while managing risk effectively.

1. Long-Term Investment Strategy:
Risk Factors:
- Market volatility
- Potential for capital loss over time
Recommendations for Risk Mitigation:
- Set clear investment goals
- Understand risk tolerance
- Regularly review investment portfolio

2. Options Trading Strategy:
Risk Factors:
- Complexity of the strategy
- Leverage
- Time decay
- Volatility
- Counterparty risk
- Early assignment
- Liquidity
Recommendations for Risk Mitigation:
- Gain thorough understanding of options trading
- Use options to hedge positions
- Be prepared for large price swings

3. Swing Trading Strategy:
Risk Factors:
- Frequency of trading
- Market volatility
- Emotional decision making
Recommendations for Risk Mitigation:
- Incorporate risk management
- Set specific conditions like stop loss
- Avoid overstaying positions

4. Diversification Strategy:
Risk Factors:
- Spreading resources too thin
- Losing focus on core business
- Lack of understanding in new markets
Recommendations for Risk Mitigation:
- Carefully allocate resources
- Maintain focus on core business
- Gain understanding of new markets

Conclusion:
While the proposed trading strategies for AAPL present various risks, implementing the recommended risk mitigation strategies can help optimize returns and manage risk effectively. By aligning execution methods with the user's risk tolerance and investment goals, informed decisions can be made to maximize profitability. It is crucial to continuously monitor and adjust strategies based on market conditions and individual preferences to achieve long-term success in trading activities.